<a href="https://colab.research.google.com/github/iamsusiep/slp2019/blob/master/get_youtube_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pulling Audio and Video from Youtube
All you should need to do to run this notebook is download the CSVs linked in the second code cell. Once done, you'll be able to download each film trailer from YouTube. Plus, before pulling a film it checks if it's already downloaded, and if it's failed previously, so that you shouldn't need to attempt pulling a film more than once.

In [0]:
!pip install youtube_dl
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import glob
import requests

     |████████████████████████████████| 1.8MB 2.8MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# load in CSVs
# first is youtube trailer links; second is IMDB ids for those movies
# third is their average ratings and num votes
# fourth is basic film metadata
youtubeIDs = pd.read_csv('/content/gdrive/My Drive/ml-youtube.csv') # https://grouplens.org/datasets/movielens/20m-youtube/
links = pd.read_csv('/content/gdrive/My Drive/links.csv') # http://files.grouplens.org/datasets/movielens/ml-20m-README.html
titleRatings = pd.read_csv('/content/gdrive/My Drive/title.ratings.tsv', sep='\t', usecols = ['tconst', 'numVotes']) # https://datasets.imdbws.com/
titleRatings['tconst'] = titleRatings['tconst'].str[2:].astype(int)
titleBase = pd.read_csv('/content/gdrive/My Drive/title.basics.tsv', sep='\t', usecols = ['tconst', 'runtimeMinutes']) # https://datasets.imdbws.com/
titleBase['tconst'] = titleBase['tconst'].str[2:].astype(int)

# merge them together
yt_links = pd.merge(youtubeIDs, links, on = 'movieId')
yt_links_ratings = pd.merge(yt_links, titleRatings, left_on='imdbId', right_on = 'tconst') 
final = pd.merge(yt_links_ratings, titleBase, left_on='tconst', right_on = 'tconst')
final = final[(final['numVotes'] > 5000) & (final['runtimeMinutes'].str.replace("\\N", "-1", regex = False).astype(int) > 40)]

In [0]:
! mkdir '/content/gdrive/My Drive/audio'

mkdir: cannot create directory ‘/content/gdrive/My Drive/audio’: File exists


In [0]:
from os.path import basename

existing_files = glob.glob("/content/gdrive/My Drive/audio/*.wav")
pulledAudio = {basename(x).split('.wav')[0]: None for x in existing_files}

In [0]:
cd '/content/gdrive/My Drive/audio'

/content/gdrive/My Drive/audio


In [0]:
try:
  fails = {line.rstrip('\n'): None for line in open('fails.txt', 'r')}
except:
  fails = {}


In [0]:
import youtube_dl
import time
# per https://stackoverflow.com/questions/27473526/download-only-audio-from-youtube-video-using-youtube-dl-in-python-script
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'postprocessor_args': [
        '-ar', '16000'
    ],
    'prefer_ffmpeg': True,
    'keepvideo': False
}

for yt_link in reversed(final['youtubeId'].values):
  if not (any([yt_link in x for x in list(pulledAudio.keys())])) and (yt_link not in fails):
    try:
      with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(['http://www.youtube.com/watch?v=' + yt_link])
    except Exception as e:
      if 'Too Many Requests' in str(e) or 'URLError' in str(e) or 'reset by peer' in str(e):
        time.sleep(10000)
        pass
      elif 'unavailable' in str(e) or 'payment' in str(e) or 'spam' in str(e) or 'nudity' in str(e) or 'terminated' in str(e) or 'contains content' in str(e) or 'Terms of Service' in str(e) or 'copyright' in str(e) or 'removed' in str(e) or 'video is private' in str(e) or 'not made this video available' in str(e):
        f = open("fails.txt","a+")
        f.write(yt_link)
        f.write('\n')
        f.close()
      else:
        pass


[youtube] 3EceNo1LJj4: Downloading webpage
[youtube] 3EceNo1LJj4: Downloading video info webpage


ERROR: This video is not available.
Sorry about that.


[youtube] kr7Opycx_yY: Downloading webpage
[youtube] kr7Opycx_yY: Downloading video info webpage


ERROR: This video is not available.
Sorry about that.


[youtube] FNppLrmdyug: Downloading webpage
[youtube] FNppLrmdyug: Downloading video info webpage
[download] Destination: Bridesmaids - Trailer-FNppLrmdyug.webm
[download] 100% of 2.07MiB in 00:03
[ffmpeg] Destination: Bridesmaids - Trailer-FNppLrmdyug.wav
Deleting original file Bridesmaids - Trailer-FNppLrmdyug.webm (pass -k to keep)
[youtube] U2p1u13R-eE: Downloading webpage
[youtube] U2p1u13R-eE: Downloading video info webpage
[download] Destination: Something Borrowed Official Trailer #1 - (2011) HD-U2p1u13R-eE.webm
[download] 100% of 2.64MiB in 00:00
[ffmpeg] Destination: Something Borrowed Official Trailer #1 - (2011) HD-U2p1u13R-eE.wav
Deleting original file Something Borrowed Official Trailer #1 - (2011) HD-U2p1u13R-eE.webm (pass -k to keep)
[youtube] RXQsSStHI5c: Downloading webpage
[youtube] RXQsSStHI5c: Downloading video info webpage
[download] Destination: SUBMARINO (2010) - Official Trailer [HD] - English subtitles-RXQsSStHI5c.webm
[download] 100% of 1.83MiB in 00:01
[ffmpe

...

ERROR: The YouTube account associated with this video has been terminated due to multiple third-party notifications of copyright infringement.
Sorry about that.


[youtube] 6tsnPmmVYx4: Downloading webpage
[youtube] 6tsnPmmVYx4: Downloading video info webpage
[download] Destination: Des hommes et des dieux _ clip #1 Cannes 2010 IN COMPETITION Xavier Beauvois-6tsnPmmVYx4.m4a
[download] 100% of 1.51MiB in 00:00
[ffmpeg] Correcting container in "Des hommes et des dieux _ clip #1 Cannes 2010 IN COMPETITION Xavier Beauvois-6tsnPmmVYx4.m4a"
[ffmpeg] Destination: Des hommes et des dieux _ clip #1 Cannes 2010 IN COMPETITION Xavier Beauvois-6tsnPmmVYx4.wav
Deleting original file Des hommes et des dieux _ clip #1 Cannes 2010 IN COMPETITION Xavier Beauvois-6tsnPmmVYx4.m4a (pass -k to keep)
[youtube] t3PuZo8qLxo: Downloading webpage
[youtube] t3PuZo8qLxo: Downloading video info webpage


ERROR: This video is unavailable.
Sorry about that.


[youtube] zqizkKDpxRE: Downloading webpage
[youtube] zqizkKDpxRE: Downloading video info webpage
[download] Destination: Whisper (2007) Official Trailer - Josh Holloway, Jennifer Shirley Movie HD-zqizkKDpxRE.webm
[download] 100% of 1.55MiB in 00:00
[ffmpeg] Destination: Whisper (2007) Official Trailer - Josh Holloway, Jennifer Shirley Movie HD-zqizkKDpxRE.wav
Deleting original file Whisper (2007) Official Trailer - Josh Holloway, Jennifer Shirley Movie HD-zqizkKDpxRE.webm (pass -k to keep)
[youtube] p_zQfVepveQ: Downloading webpage
[youtube] p_zQfVepveQ: Downloading video info webpage
[download] Destination: Borderland (2007) Trailer Ingles-p_zQfVepveQ.webm
[download] 100% of 2.00MiB in 00:00
[ffmpeg] Destination: Borderland (2007) Trailer Ingles-p_zQfVepveQ.wav
Deleting original file Borderland (2007) Trailer Ingles-p_zQfVepveQ.webm (pass -k to keep)
[youtube] rwBaS6m3q5c: Downloading webpage
[youtube] rwBaS6m3q5c: Downloading video info webpage
[download] Destination: EYES WIDE OPEN 

ERROR: Unable to download webpage: HTTP Error 429: Too Many Requests (caused by <HTTPError 429: 'Too Many Requests'>); please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.
